In [0]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


import torch
import torchvision
import torchvision.transforms as transforms


import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


%matplotlib inline

In [0]:
transform = transforms.Compose(
    [transforms.CenterCrop((28,28)),transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
# transform = transforms.Compose(
#     [transforms.ToTensor(),transforms.CenterCrop(28,28)])

In [33]:
cifar_trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
cifar_testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# train_split_len = 26316
# # test_split_len 
# part_train = torch.utils.data.random_split(cifar_trainset, [train_split_len, len(cifar_trainset)-train_split_len])[0]
# # part_te = torch.utils.data.random_split(cifar_testset, [test_split_len, len(cifar_testset)-test_split_len])[0]
# len(part_train)


In [35]:
cifar_trainset_ran = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

Files already downloaded and verified


In [36]:
corruption_percentage = 0.5
epochs = 55
train_split_len = 25000 + int(25000*corruption_percentage*100/(100-corruption_percentage*100))
print(train_split_len)
cifar_trainset_random = torch.utils.data.random_split(cifar_trainset_ran, [train_split_len, len(cifar_trainset)-train_split_len])[0]
cifar_trainset_ran.data = cifar_trainset_ran.data[cifar_trainset_random.indices]
tgts = np.array(cifar_trainset_ran.targets)
cifar_trainset_ran.targets = tgts[cifar_trainset_random.indices]

50000


In [37]:
cifar_trainset_ran.data.shape

(50000, 32, 32, 3)

In [38]:

cifar_trainset_random = cifar_trainset_ran
print(len(cifar_trainset_random.targets))

np.random.seed(1234)
mask = np.random.uniform(0,1, train_split_len) < corruption_percentage

a = np.array(cifar_trainset_random.targets)
print(len(a))
print("true",a[mask])
a[mask] = np.random.randint(0,10,sum(mask))
print("randomized",a[mask])
cifar_trainset_random.targets = list(a)
cifar_trainset_random.targets
# cifar_trainset_random.targets[:50000] = np.random.randint(low=0,high=9,size=50000)
trainloader_random = torch.utils.data.DataLoader(cifar_trainset_random,batch_size=256,shuffle=False,num_workers=2)

50000
50000
true [2 5 1 ... 0 6 3]
randomized [9 1 5 ... 4 1 5]


In [39]:
cifar_trainset_random.data.shape

(50000, 32, 32, 3)

In [40]:
np.unique(cifar_trainset.targets),sum(mask)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), 24980)

In [0]:

trainloader = torch.utils.data.DataLoader(cifar_trainset, batch_size=256,
                                          shuffle=False, num_workers=2)
testloader = torch.utils.data.DataLoader(cifar_testset, batch_size=256,
                                         shuffle=False, num_workers=2)


In [0]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [0]:
dataiter = iter(trainloader_random)
images, labels = dataiter.next()


In [44]:
images.shape

torch.Size([256, 3, 28, 28])

In [0]:
class Conv_module(nn.Module):
    def __init__(self,inp_ch,f,s,k,pad):
        super(Conv_module,self).__init__()
        self.inp_ch = inp_ch
        self.f = f
        self.s = s 
        self.k = k 
        self.pad = pad
        
        
        self.conv = nn.Conv2d(self.inp_ch,self.f,k,stride=s,padding=self.pad)
        self.bn = nn.BatchNorm2d(self.f)
        self.act = nn.ReLU()
    def forward(self,x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.act(x)
        return x

In [46]:
conv = Conv_module(3,64,1,3,1)
conv.forward(images).shape

torch.Size([256, 64, 28, 28])

In [0]:
class inception_module(nn.Module):
    def __init__(self,inp_ch,f0,f1):
        super(inception_module, self).__init__()
        self.inp_ch = inp_ch
        self.f0 = f0
        self.f1 = f1
        
        
        
        self.conv1 = Conv_module(self.inp_ch,self.f0,1,1,pad=0)
        self.conv3 = Conv_module(self.inp_ch,self.f1,1,3,pad=1)
        #self.conv1 = nn.Conv2d(3,self.f0,1)
        #self.conv3 = nn.Conv2d(3,self.f1,3,padding=1)
    def forward(self,x):
        x1 = self.conv1.forward(x)
        x3 = self.conv3.forward(x)
        #print(x1.shape,x3.shape)
        
        x = torch.cat((x1,x3),dim=1)
        
    
        return x

In [48]:
inc_module = inception_module(96,32,32)
conv_module = Conv_module(3,96,1,1,0)
out = conv_module(images)
inc_module.forward(out).shape

torch.Size([256, 64, 28, 28])

In [0]:
class downsample_module(nn.Module):
    def __init__(self,inp_ch,f):
        super(downsample_module,self).__init__()
        self.inp_ch = inp_ch
        self.f = f
        self.conv = Conv_module(self.inp_ch,self.f,2,3,pad=0)
        self.pool = nn.MaxPool2d(3,stride=2,padding=0)
    def forward(self,x):
        x1 = self.conv(x)
        #print(x1.shape)
        x2 = self.pool(x)
        #print(x2.shape)
        x = torch.cat((x1,x2),dim=1)
        
        return x,x1

In [0]:
class inception_net(nn.Module):
    def __init__(self):
        super(inception_net,self).__init__()
        self.conv1 = Conv_module(3,96,1,3,0)
        
        self.incept1 = inception_module(96,32,32)
        self.incept2 = inception_module(64,32,48)
        
        self.downsample1 = downsample_module(80,80)
        
        self.incept3 = inception_module(160,112,48)
        self.incept4 = inception_module(160,96,64)
        self.incept5 = inception_module(160,80,80)
        self.incept6 = inception_module(160,48,96)
        
        self.downsample2 = downsample_module(144,96)
        
        self.incept7 = inception_module(240,176,60)
        self.incept8 = inception_module(236,176,60)
        
        self.pool = nn.AvgPool2d(5)
        
        
        
        self.linear = nn.Linear(236,10)
    def forward(self,x):
        x = self.conv1.forward(x)
        #act1 = x
        
        x = self.incept1.forward(x)
        #act2 = x
        
        x = self.incept2.forward(x)
        #act3 = x
        
        x,act4 = self.downsample1.forward(x)
        
        x = self.incept3.forward(x)
        #act5 = x
        
        x = self.incept4.forward(x)
        #act6 = x
        
        x = self.incept5.forward(x)
        #act7 = x
        
        x = self.incept6.forward(x)
        #act8 = x
        
        x,act9 = self.downsample2.forward(x)
        
        x = self.incept7.forward(x)
        #act10 = x
        x = self.incept8.forward(x)
        #act11 = x
        
        #print("outs before pooling",x.size())
        x = self.pool(x)
        #print("hello")
        #print("outs after pooling",x.size())
        x = x.view(-1,1*1*236)
        x = self.linear(x) 
        
        #print("outs",x)
        #activatn = {"act1":act1,"act2":act2,"act3":act3,"act4":act4,"act5":act5,"act6":act6,
         #           "act7":act7,"act8":act8,"act9":act9,"act10":act10,"act11":act11}
        return x
        
        
    

In [0]:
inc_net_random_labels = inception_net()
#inc_net_random_labels(images).shape
inc_net_random_labels = inc_net_random_labels.to("cuda")

In [0]:

criterion_inc_rand = nn.CrossEntropyLoss()
optimizer_inc_rand = optim.SGD(inc_net_random_labels.parameters(), lr=0.01, momentum=0.9)

In [53]:
actri = []
lossr_curi = []
inc_net_random_labels.train()
# epochs = 30
for epoch in range(epochs):  # loop over the dataset multiple times
    ep_lossri = []
    

    running_loss = 0.0
    for i, data in enumerate(trainloader_random, 0):
        # get the inputs
        inputs, labels = data
        #print(inputs.size())
        inputs,labels = inputs.to("cuda"),labels.to("cuda")

        # zero the parameter gradients
        optimizer_inc_rand.zero_grad()

        # forward + backward + optimize
        outputs = inc_net_random_labels(inputs)
        
#         print(outputs.shape)
        
        loss = criterion_inc_rand(outputs, labels)
        loss.backward()
        optimizer_inc_rand.step()

        # print statistics
        running_loss += loss.item()
        if i % 50 == 49:    # print every 50 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 50))
            ep_lossri.append(running_loss)
            running_loss = 0.0
            
    lossr_curi.append(np.mean(ep_lossri))   #loss per epoch
#     if (epoch%5 == 0):
#         _,actirs= inc(inputs)
#         actri.append(actirs)
    

print('Finished Training')


[1,    50] loss: 2.220
[1,   100] loss: 2.143
[1,   150] loss: 2.108
[2,    50] loss: 2.059
[2,   100] loss: 2.044
[2,   150] loss: 2.022
[3,    50] loss: 1.987
[3,   100] loss: 1.986
[3,   150] loss: 1.968
[4,    50] loss: 1.932
[4,   100] loss: 1.940
[4,   150] loss: 1.919
[5,    50] loss: 1.882
[5,   100] loss: 1.894
[5,   150] loss: 1.871
[6,    50] loss: 1.830
[6,   100] loss: 1.847
[6,   150] loss: 1.826
[7,    50] loss: 1.776
[7,   100] loss: 1.812
[7,   150] loss: 1.783
[8,    50] loss: 1.727
[8,   100] loss: 1.774
[8,   150] loss: 1.743
[9,    50] loss: 1.681
[9,   100] loss: 1.705
[9,   150] loss: 1.708
[10,    50] loss: 1.642
[10,   100] loss: 1.639
[10,   150] loss: 1.655
[11,    50] loss: 1.573
[11,   100] loss: 1.557
[11,   150] loss: 1.591
[12,    50] loss: 1.502
[12,   100] loss: 1.492
[12,   150] loss: 1.564
[13,    50] loss: 1.440
[13,   100] loss: 1.417
[13,   150] loss: 1.506
[14,    50] loss: 1.377
[14,   100] loss: 1.347
[14,   150] loss: 1.421
[15,    50] loss: 1

In [0]:
torch.save(inc_net_random_labels.state_dict(),"/content/drive/My Drive/Research/CIFAR Corrupted vs uncorrupted/Inception net weights/Weights_when_train_on_equal_true_data/model_epoch"+str(epochs)+"_"+str(corruption_percentage)+".pt")

In [0]:
# inc_net_random_labels.load_state_dict(torch.load("/content/drive/My Drive/Research/CIFAR Corrupted vs uncorrupted/Inception net weights/Weights_when_train_on_equal_true_data/model_epoch1_0.1.pt"))

In [56]:
correct = 0
total = 0
inc_net_random_labels.eval()
with torch.no_grad():
    for data in trainloader_random:
        images, labels = data
        images,labels = images.to("cuda"),labels.to("cuda")
        outputs = inc_net_random_labels(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the %d train images: %d %%' % (train_split_len, 100 * correct / total))

Accuracy of the network on the 50000 train images: 100 %


In [57]:
correct = 0
total = 0
loss = 0
batch = 0
inc_net_random_labels.eval()
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images,labels = images.to("cuda"),labels.to("cuda")
        outputs = inc_net_random_labels(images)
        
        loss += criterion_inc_rand(outputs, labels)
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        batch+=1

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))
print("loss", loss/batch)

Accuracy of the network on the 10000 test images: 42 %
loss tensor(3.7541, device='cuda:0')


In [0]:
def accuracy( a, b):
  length = a.shape
  correct = a==b
  return sum(correct)/length

In [59]:
correct = 0
total = 0
train_loss = 0
true = []
pred=[]
out =[]

inc_net_random_labels.eval()

with torch.no_grad():
    for data in trainloader_random:
        images, labels = data
        images,labels = images.to("cuda"),labels.to("cuda")
        true.append(labels.cpu().numpy())
        
        outputs = inc_net_random_labels(images)
        out.append(outputs.cpu())
        loss = criterion_inc_rand(outputs, labels)
        
        train_loss += loss.item()
        
        _, predicted = torch.max(outputs.data, 1)
        pred.append(predicted.cpu().numpy())
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the %d train images: %d %%' % (train_split_len, 100 * correct / total))
true_targets = np.concatenate(true,axis=0)
predicted_targets = np.concatenate(pred,axis =0)

print("---"*20)
print("Train accuracy on corrupt data",accuracy(true_targets[mask], predicted_targets[mask]))
print("Train accuracy on un-corrupt data",accuracy(true_targets[~mask], predicted_targets[~mask]))
print("Train accuracy on full data", accuracy(true_targets, predicted_targets))

# print( sum (predicted_targets == np.argmax(out, axis =1)))

l= np.where(mask ==True)
p = np.where(mask == False)

out = torch.cat(out, dim =0)

print("Train cross entropy loss on corrupt data", criterion_inc_rand(out[l], torch.Tensor(true_targets[l]).type(torch.LongTensor)))
print("Train cross entropy loss on un-corrupt data",criterion_inc_rand(out[p], torch.Tensor(true_targets[p]).type(torch.LongTensor)))
print("Train cross entropy loss on full data",criterion_inc_rand(out, torch.Tensor(true_targets).type(torch.LongTensor)))
print("---"*20)

Accuracy of the network on the 50000 train images: 100 %
------------------------------------------------------------
Train accuracy on corrupt data [1.]
Train accuracy on un-corrupt data [1.]
Train accuracy on full data [1.]
Train cross entropy loss on corrupt data tensor(0.0014)
Train cross entropy loss on un-corrupt data tensor(0.0008)
Train cross entropy loss on full data tensor(0.0011)
------------------------------------------------------------


In [60]:
testset=cifar_testset
testset_len = len(testset.targets)
np.random.seed(1234)
mask1 = np.random.uniform(0,1,testset_len) < corruption_percentage

a = np.array(testset.targets)
# print("true",a[mask1])
print(np.sum(mask1))
a[mask1] = np.random.randint(0,10,sum(mask1))
# print("randomized",a[mask1])
testset.targets = list(a)


correct = 0
total = 0
test_loss = 0
true = []
pred=[]
out =[]
inc_net_random_labels.eval()
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images,labels = images.to("cuda"),labels.to("cuda")
        true.append(labels.cpu().numpy())
                
        outputs = inc_net_random_labels(images)
        out.append(outputs.cpu())
        loss = criterion_inc_rand(outputs, labels)
        
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        pred.append(predicted.cpu().numpy())
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))
true_targets = np.concatenate(true,axis=0)
predicted_targets = np.concatenate(pred,axis =0)
print("---"*20)
print("Test accuracy on corrupt data",accuracy(true_targets[mask1], predicted_targets[mask1]))
print("Test accuracy on un-corrupt data",accuracy(true_targets[~mask1], predicted_targets[~mask1]))
print("Test accuracy on full data", accuracy(true_targets, predicted_targets))

l = np.where(mask1 ==True)
p  = np.where(mask1 == False)
out = torch.cat(out, dim =0)

print("Test cross entropy loss on corrupt data", criterion_inc_rand(out[l], torch.Tensor(true_targets[l]).type(torch.LongTensor)))
print("Test cross entropy loss on un-corrupt data",criterion_inc_rand(out[p], torch.Tensor(true_targets[p]).type(torch.LongTensor)))
print("Test cross entropy loss on full data",criterion_inc_rand(out, torch.Tensor(true_targets).type(torch.LongTensor)))
print("---"*20)

4958
Accuracy of the network on the 10000 test images: 26 %
------------------------------------------------------------
Test accuracy on corrupt data [0.10346914]
Test accuracy on un-corrupt data [0.42542642]
Test accuracy on full data [0.2658]
Test cross entropy loss on corrupt data tensor(8.9452)
Test cross entropy loss on un-corrupt data tensor(3.7132)
Test cross entropy loss on full data tensor(6.3072)
------------------------------------------------------------
